In [ ]:
import numpy as np
import pandas as pd
import sqlite3 # SELECT sql FROM sqlite_master WHERE name='foo';
import csv
import hashlib
import glob
import re
import os
import unidecode

pd.set_option('display.max_columns', 1000)
# pd.set_option('display.max_rows', 300)
# pd.set_option('auto_truncate_string_exceeding_this_length', 1000)
pd.set_option('max_colwidth', 400)

def toNumber(x):
    try:
        return float(x.replace('\'', '').replace(',', '.'))
    except Exception as e:
        pass # print(e)
    return float(x)

def hashDateDescription(x):
    return x[0].strftime("%Y-%m-%d-") + hashlib.md5((x[1]+str(x[2])).encode('utf-8')).hexdigest()[:8]

def hashBankDescription(x):
    val = ''.join([str(x) for x in x])
    return hashlib.md5(val.encode('utf-8')).hexdigest()[:16]


In [ ]:
import gspread
# https://github.com/uehara1414/df2gspread
from df2gspread import df2gspread as d2g
from df2gspread import gspread2df as g2d
from oauth2client.service_account import ServiceAccountCredentials

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name('finance-sheets-089599011f90.json', scope)
gc = gspread.authorize(credentials)

In [ ]:
import collections
from abc import ABC, abstractmethod

class Rule(ABC):
    @abstractmethod
    def Match(description):
        pass

class ExactRule(Rule):
    pass

class RegexRule(Rule):
    pass

class Category():
    def __init__(self, name, index, parent_category=None):
        self.name = name
        self.index = index
        self.parent_category = parent_category
        self.rules = []
        self.subcategories = []
    
    def __getitem__(self, sliced):
        return self.subcategories[sliced]
    
    def __str__(self, subcategories=True, rules=True, numbered=False, prefix=''):
        ret = []
        
        name = prefix
        if numbered: name += str(self.index) + '. '
        name += self.name
        ret.append(name)
        
        if subcategories:
            ret.extend([category.__str__(
            subcategories=subcategories,
            rules=rules,
            numbered=numbered,
            prefix='\t'
        ) for category in self.subcategories])
        
        return '\n'.join(ret)
    
    def AddSubCategory(self, name, index):
        subcategory = Category(name, index, self)
        self.subcategories.append(subcategory)
        return subcategory
        
    def AddRule(self, rule):
        self.rules.append(rule)
    
    def ListRules(self, prefix=''):
        return '\n'.join([prefix+str(rule) for rule in self.rules])
    
    def _formatRule(self, rule, prefix=''):
        return str(prefix) + str(rule)

    def _formatCategory(self, index, numbered, prefix=''):
        index = '' if not numbered else str(index) + '. '
        return str(prefix) + str(index) + self.name

class Categories():
    def __init__(self, filename):
        categories = []
        with open(filename) as f:
            num_categories = 0
            for line in f:
                line = line.rstrip()
                if not line: continue
                name = line.strip()
                
                if line[0] == '\t':
                    categories[-1].AddSubCategory(name, num_subcategories)
                    num_subcategories += 1
                else:
                    category = Category(name, num_categories)
                    num_categories += 1
                    num_subcategories = 0
                    categories.append(category) 
        self.categories = categories
        
    def __str__(self):
        return self.list(True)
    
    def __getitem__(self, sliced):
        return self.categories[sliced]
    
    def Iterate(self, main=True, sub=True):
        for c in self.categories:
            if main: yield c
            
            if not sub: continue
            for cc in c.subcategories:
                yield cc
                
    def Save(self, filename):
        with open(filename, 'w') as f:
            f.write('\n'.join([x.__str__() for x in self]))

categories = Categories('categories.txt')
# categories.Save('test.txt')
# print('\n'.join([x.__str__(subcategories=True, rules=True, numbered=True) for x in categories]))
# print('\n'.join([x.__str__(subcategories=False, rules=True, numbered=True) for x in categories.Iterate(False)]))

a = []
for c in categories:
    a.append((None, c.name))
    
    for sc in c:
        a.append((sc.parent_category.index, sc.name))

conn = sqlite3.connect("transactions.db")
apd = pd.DataFrame(a)
apd.columns = ['ParentCategoryId', 'Name']
apd.to_sql("Categories", conn, if_exists="replace", index_label='CategoryId')

apd

In [ ]:
from enum import Enum
class Bank(Enum):
    FIO = 1
    UBS_transaction = 2
    UBS_export = 3
    CHASE = 4

def DetectEncoding(filename, read_lines=20):
    encodings = ['utf-8-sig', 'utf-8', 'windows-1250']

    for encoding in encodings:
        try:
            with open(filename, encoding=encoding) as f:
                return encoding, [next(f) for x in range(read_lines)]
        except StopIteration as e:
            with open(filename, encoding=encoding) as f:
                return encoding, f.readlines()
        except UnicodeDecodeError as e:
            pass
    raise Error('Unknown Encoding for file {}'.format(filename))
    
def GuessBank(filename):
    encoding, lines = DetectEncoding(filename)

    if lines[0] == 'sep=;\n' and lines[1] == 'Account number;Card number;Account/Cardholder;Purchase date;Booking text;Sector;Amount;Original currency;Rate;Currency;Debit;Credit;Booked\n':
        return encoding, Bank.UBS_transaction
    if lines[0] == 'Valuation date;Banking relationship;Portfolio;Product;IBAN;Ccy.;Date from;Date to;Description;Trade date;Booking date;Value date;Description 1;Description 2;Description 3;Transaction no.;Exchange rate in the original amount in settlement currency;Individual amount;Debit;Credit;Balance\n':
        return encoding, Bank.UBS_export
    if lines[0] == 'Details,Posting Date,Description,Amount,Type,Balance,Check or Slip #\n':
        return encoding, Bank.CHASE
    if lines[14] == 'ID pohybu;Datum;Objem;Měna;Protiúčet;Název protiúčtu;Kód banky;Název banky;KS;VS;SS;Uživatelská identifikace;Zpráva pro příjemce;Typ;Provedl;Upřesnění;Komentář;BIC;ID pokynu\n':
        return encoding, Bank.FIO
    
    print(lines[:2])
    raise Warning('Could not detect Bank for file {}'.format(filename))

def ParseFIO(filename, encoding):
    # Parse Metadata
    with open(filename, encoding=encoding) as f:
        metadata = dict(next(f).strip().split(';') for x in range(13))
    
    # ID pohybu, Datum, Objem, Měna, Protiúčet, Název protiúčtu, Kód banky, 
    # Název banky, KS, VS, SS, Uživatelská identifikace, Zpráva pro příjemce,
    # Typ, Provedl, Upřesnění, Komentář, BIC, ID pokynu
    df = pd.read_csv(filename, skiprows=14, sep=';', index_col=False, dtype=str, encoding=encoding)

    df['Uživatelská identifikace'] = df['Uživatelská identifikace'].fillna('')
    df['Zpráva pro příjemce'] = df['Zpráva pro příjemce'].fillna('')
    df['Komentář'] = df['Komentář'].fillna('')
    df['Amount'] = df['Objem'].apply(toNumber).fillna(0)
    
    return pd.DataFrame({
        'TransactionId': df['ID pohybu'],
        'PurchaseDate': df['Datum'].apply(pd.to_datetime, format="%d.%m.%Y", errors='coerce'),
        'Amount': df['Amount'],
        'Currency': df['Měna'],
        'Description': df[['Zpráva pro příjemce', 'Uživatelská identifikace', 'Komentář']].apply(lambda x: ' '.join(pd.unique(x)), axis=1),
        'Balance': df['Amount'].cumsum() + toNumber(metadata['openingBalance']),
        'TransactionType': df['Typ'].fillna('')
    }).iloc[::-1].sort_values(by='PurchaseDate', ascending=False, kind='mergesort')

def ParseCHASE(filename, encoding):
    # Details, Posting Date, Description, Amount, Type, Balance, Check or Slip
    df = pd.read_csv(filename, skiprows=0, sep=',', index_col=False, dtype=str, encoding=encoding)

    df['Description'] = df['Description'].fillna('')
    df['PurchaseDate'] = df['Posting Date'].apply(pd.to_datetime, format="%m/%d/%Y", errors='coerce')
    
    return pd.DataFrame({
        'TransactionId': df[['PurchaseDate', 'Description', 'Amount']].apply(hashDateDescription, axis=1),
        'PurchaseDate': df['PurchaseDate'],
        'Amount': df['Amount'].apply(toNumber).fillna(0),
        'Currency': 'USD',
        'Description': df['Description'],
        'Balance': df['Balance'].apply(toNumber).fillna(0),
    })

def ParseUBS_transactions(filename, encoding):
    # Account number, Card number, Account/Cardholder, Purchase date,Booking text,
    # Sector, Amount, Original currency, Rate, Currency, Debit, Credit, Booked
    df = pd.read_csv(filename, skiprows=1, sep=';', index_col=False, dtype=str, encoding=encoding)

    df['AmountCalculated'] = df['Credit'].apply(toNumber).fillna(0) - df['Debit'].apply(toNumber).fillna(0)
    df['PurchaseDate'] = df['Purchase date'].apply(pd.to_datetime, format="%d.%m.%Y")
    df['Description'] = df['Booking text'].fillna('')

    # Remove rows defined by descriptionsToRemove
    descriptionsToRemove = ['Balance brought forward', 'DIRECT DEBIT', 'Total per currency', 'Total card bookings']
    matchingDescriptions = df['Description'].str.contains('|'.join(descriptionsToRemove))
    noCardNumber = df['Card number'].isnull()
    df.drop(df[matchingDescriptions & noCardNumber].index, inplace=True)
    
    return pd.DataFrame({
        'TransactionId': df[['PurchaseDate', 'Description', 'AmountCalculated']].apply(hashDateDescription, axis=1),
        'PurchaseDate': df['PurchaseDate'],
        'Amount': df['AmountCalculated'],
        'Currency': df['Currency'],
        'Description': df['Description'],
        # Extra
        'ExcRateToCHF': df['Rate'].apply(toNumber).fillna(0),
        'OriginalAmount': df['Amount'].apply(toNumber),
        'OriginalCurrency': df['Original currency'],
        'UbsCategory': df['Sector'],
    }).iloc[::-1].sort_values(by='PurchaseDate', ascending=False, kind='mergesort')

def ParseUBS_export(filename, encoding):
    # Valuation date, Banking relationship, Portfolio, Product, IBAN, Ccy.,
    # Date from, Date to, Description, Trade date, Booking date, Value date,
    # Description 1, Description 2, Description 3, Transaction no.,
    # Exchange rate in the original amount in settlement currency,
    # Individual amount, Debit, Credit, Balance
    df = pd.read_csv(filename, skiprows=0, sep=';', index_col=False, dtype=str, encoding=encoding)
    df = df.iloc[:-3] # remove last three rows: 'Summary'

    df['Debit'] = df['Debit'].apply(toNumber).fillna(0)
    df['Credit'] = df['Credit'].apply(toNumber).fillna(0)
    df['Individual amount'] = df['Individual amount'].apply(toNumber).fillna(0)
    df['Description 2'] = df['Description 2'].fillna('')
    df['Description 3'] = df['Description 3'].fillna('')

    df = pd.DataFrame({
        'TransactionId': df['Transaction no.'],
        'PurchaseDate': df['Trade date'].apply(pd.to_datetime, format="%d.%m.%Y"),
        'Amount': df['Credit'] - df['Debit'] + df['Individual amount'],
        'Currency': df['Ccy.'],
        'Description': df[['Description 2', 'Description 3']].apply(lambda x: '\n'.join(x), axis=1),
        'Balance': df['Balance'].apply(toNumber).fillna(0),
        # Extra
        'ExcRateToCHF': df['Exchange rate in the original amount in settlement currency'],
        'TransactionType': df['Description 1'],
    })

    # Differentiate Multi PayNet Orders
    tid = df[df.TransactionType=='Multi PayNet Order']
    df.drop(tid.index, inplace=True) # Drop Sum rows
    filtered = df[df.TransactionId.isin(tid.TransactionId)]
    df.loc[filtered.index, 'Description'] = filtered.TransactionType + '\n' + filtered.Description
    df.loc[filtered.index, 'TransactionType'] = 'Multi PayNet Order'
    # Create new IDs by appending numbers
    decreasingSeries = pd.Series(range(len(filtered.index)-1,-1,-1)).astype(str)
    newTransactionIds = filtered.TransactionId.str.cat(decreasingSeries)
    df.loc[filtered.index, 'TransactionId'] = newTransactionIds

    # Group by UBS bank charges
    mapping = {col:'first' for col in df.columns.values}
    mapping['Amount'] = 'sum'
    df = df.groupby('TransactionId', sort=False).agg(mapping).reset_index(drop=True)
    df.loc[df['TransactionType']=='Balance of service prices', 'Description'] = 'UBS charges'
    
    # Remove Direct Debit since we have Credit Card transactions
    df.drop(df[df['TransactionType'] == 'Direct Debit'].index, inplace=True)
    
    return df

def GroupByTransactionIdRemoveEmpty(df):
    # Group by TransactionId: joins together credit card fees
    mapping = {col:'first' for col in df.columns.values}
    mapping['Amount'] = 'sum'
    df = df.groupby('TransactionId', sort=False).agg(mapping)

    # Remove transactions with 0 amount
    df.drop(df[df.Amount==0].index, inplace=True)
    return df

def CommonPostprocessing(df, path, bank):
    dirs, file = os.path.split(path)
    dirs, last_folder = os.path.split(dirs)

    df = GroupByTransactionIdRemoveEmpty(df)

    # Add metadata
    df['AccountFolder'] = last_folder
    df['Filename'] = file
    df['BankEnum'] = bank
    
    df['OriginalDescription'] = df['Description']
    df['Description'] = df['Description'].apply(CleanText)
    
    return df

def CommonPostprocessingAfterJoin(df):
#     df = GroupByTransactionIdRemoveEmpty(df)
    df.drop_duplicates(subset=['PurchaseDate','TransactionId'], inplace=True)
    df.sort_values(by='PurchaseDate', ascending=False, kind='mergesort', inplace=True)
    return df.reset_index(drop=True)

def LoadTransactions(filename):
    encoding, bank = GuessBank(filename)
    
    bankToParse = {
        Bank.FIO: ParseFIO,
        Bank.UBS_transaction: ParseUBS_transactions,
        Bank.UBS_export: ParseUBS_export,
        Bank.CHASE: ParseCHASE,
    }
    
    transactions = bankToParse[bank](filename, encoding)
    
    return CommonPostprocessing(transactions, filename, str(bank)[5:])

def LoadAllTransactions(folder, regex = '*'):
    excluded = [
        "Revolut",
        "U2400299.2018.dividends",
    ]
    dfs = []
    csv_files = glob.glob(os.path.join(folder + "/**/" + regex + ".csv"), recursive=True)
    for filename in csv_files:
#         print(filename)
        if (any(x in filename for x in excluded)):
            continue
        try:
            dfs.append(LoadTransactions(filename))
        except Exception as e:
            print(e)
            print(filename)
            raise e
    return CommonPostprocessingAfterJoin(pd.concat(dfs, copy=False, ignore_index=True))

def CleanText(text):
    text = text.lower()
    
    text = text.replace('\n', ' ')
    text = unidecode.unidecode(text)
    text = re.sub(r'[^\x00-\x7f]',r' ', text) # nonprintable characters  
    text = re.sub(r'[^\w ]',r' ', text)
    text = re.sub(r'\d',r' ', text)
    text = re.sub(r'(^| )\w( |$)', '', text)
    text = re.sub('[ \t\n]+', ' ', text)
#     text = re.sub('\d\S*', '', text)
    return text.strip()


In [ ]:
filename = 'transactions/2017_12_invoice_17_03_2018 18_52_50.csv'
# df = pd.read_csv(filename, skiprows=1, sep=';', index_col=False, dtype=str, encoding='windows-1250')
df = LoadTransactions(filename)
df
# df.groupby(['AccountFolder', df.PurchaseDate.dt.to_period("M")]).sum()

In [ ]:
filename = 'transactions/Chase3885_Activity_20180403.CSV'
# df = pd.read_csv(filename, skiprows=0, sep=',', encoding='UTF-8', index_col=False)
df = LoadTransactions(filename)
df

In [ ]:
filename = r'C:/Users/svecon/Google Drive/Finance\Statements\Fio pracovni\2018_06_statement.csv'
df = pd.read_csv(filename, skiprows=13, sep=';', index_col=False, dtype=str)
# df = LoadTransactions(filename)
df

In [ ]:
filename = r'C:/Users/svecon/Google Drive/Finance/Statements/Fio pracovni/2017_09_statement.csv'
# df = pd.read_csv(filename, skiprows=0, sep=';', encoding='UTF-8', index_col=False).dropna(thresh=10)
# df.groupby(['Transaction no.']).agg({'Description 1': 'first', 'Individual amount': np.sum})
# df[df['Exchange rate in the original amount in settlement currency'].notnull()]
# df
df = ParseFIO(filename, "utf-8")
df

In [ ]:
folder = 'C:\\Users\\svecon\\Google Drive\\Finance\\Statements'
def CalculateBalances(folder, output_file='Balances.csv'):
    df = LoadAllTransactions(folder)
    
    # Group by Months and Account
    df = df.groupby([df.PurchaseDate.dt.to_period("M"), 'AccountFolder'], sort=False)
    df = df.agg({'Balance': 'first', 'Amount': 'sum'})
    
    # Fill missing data
    df.sort_index(ascending=False, kind='mergesort', inplace=True)
    bal = df['Balance'].unstack(level=0).transpose().fillna(method='ffill').transpose()
    
    # Reverse order of columns (dates)
    bal = bal.loc[:,::-1]
    
    # Save to file
    bal.to_csv(output_file, sep='\t')

    return bal

CalculateBalances(folder)

In [ ]:
folder = 'C:\\Users\\svecon\\Google Drive\\Finance\\Statements\\'
df = LoadAllTransactions(folder, '*')
# df.to_csv('all_transactions.csv', sep='\t')
df = df.round(decimals=2)
df = df.replace('', np.nan)
df.info()

In [ ]:
sheet = gc.open('Expenses')
d2g.upload(df.fillna(''), sheet.id, '2019', credentials=credentials, row_names=False, value_input_opt='USER_ENTERED')

In [ ]:
dfsheet = g2d.download(gfile=sheet.id, col_names=True, row_names=False, credentials=credentials)
orig = dfsheet.dtypes.to_dict()
orig.update(df.dtypes.to_dict())
dfsheet = dfsheet.replace('', np.nan).apply(lambda x: x.astype(orig[x.name]))
dfsheet.info()

In [ ]:
keys = list(df.columns)
inner = pd.merge(df, dfsheet, on=keys, how='inner')
inner.shape

In [ ]:
pd.concat([inner,df]).drop_duplicates(keep=False)

In [ ]:
df['Description']

In [ ]:
dfsheet['Description']